### Imports & Settings
- Descargar e importar dependencias

In [238]:
# Load data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Dummy baseline
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, r2_score, mean_squared_error

# Preprocessing (Pipelines)
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Modeling: Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Modeling: Random Forest
from sklearn.ensemble import RandomForestClassifier

# Modeling: LightGBM
# Suprimir warnings (LGBM suele ser sucio con las warnings)
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", message=".*Unknown parameter.*")
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV

# Modeling: XGBoost
!pip install xgboost
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score

# Deployment
import joblib

### Load Data
- Cargar el data set
- Primer vistazo a la distribucion

In [243]:
# Get url from file hosted on GitHub
url = "https://raw.githubusercontent.com/ZocoMacc/car-quirks-ml_InnovaLab25/refs/heads/main/data/cars_data.csv"

# Cargar csv en un DataFrame
df = pd.read_csv(url)

# Explorar la estructura del DataFrame (Sanity Check)
df.info()
df.shape
# df.isna().sum().sort_values(ascending=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   name                    10000 non-null  object 
 1   year                    10000 non-null  int64  
 2   selling_price           10000 non-null  int64  
 3   km_driven               10000 non-null  int64  
 4   fuel                    10000 non-null  int64  
 5   combustible_estimado_l  10000 non-null  float64
 6   seller_type             10000 non-null  int64  
 7   transmission            10000 non-null  int64  
 8   owner                   10000 non-null  int64  
 9   tipo_carroceria         10000 non-null  int64  
 10  potencia_motor_hp       10000 non-null  int64  
 11  nivel_seguridad         10000 non-null  float64
 12  calidad_auto            10000 non-null  object 
 13  score_calidad           10000 non-null  float64
 14  eficiencia_km_l         10000 non-null 

(10000, 15)

In [ ]:
# Ver el balance de las clases (84% Media, 10.5% Alta, 5.5% Baja)
df["calidad_auto"].value_counts(normalize=True)

,proportion
calidad_auto,
Media,0.8399
Alta,0.1052
Baja,0.0549


##### Lo que esto nos dice
- Un fuerte desbalanceo de clases
  - "Media" es la mayoria con un 84%
  - "Alta" solo cubre el 10%, y "Baja" solo un 5.5%
- El accuracy de referencia (baseline) es engañosamente alto
  - Un DummyClassifier que siempre predice "Media" deberia de poder predecir 83.99%.
  - Superar este baseline no garantiza que el modelo sea mas precizo detectando "Alta" o "Baja".
- Riesgo de abandono de las clases minoritarias
  - El modelo tiene el riesgo de predecir "Media" la mayoria del tiempo debido al desbalanceo de las clases, lo que podria ignorar "Alta" o "Baja" como opciones.

### Cleaning & Dropping
- Decidir las variables que seran ignoradas por el modelo.
- Si es necesarion, transformar valores en interpretaciones legibles por el modelo o marcarlas con una categoria.

In [244]:
# Duda sobre la variable "score_calidad": Que tanto se correlaciona con "calidad_auto"
# Mapear las categorias con numeros para calcular la correlacion
mapping = {"Baja": 0, "Media": 1, "Alta": 2}
df["calidad_code"] = df["calidad_auto"].map(mapping)

# Calcular la correlacion
corr = df[["score_calidad", "calidad_code"]].corr().iloc[0, 1]

# Imprimir los resultados
print(f"Correlacion entre score_calidad y calidad_auto: {corr:.3f}")

# Ver correlaciones para identificar leaks
corr_X = df.drop(columns=["name"])
correlations = corr_X.drop(columns=["calidad_auto"]).corrwith(corr_X["calidad_code"])
print("Correlaciones")
print(correlations.sort_values(ascending=False))

Correlacion entre score_calidad y calidad_auto: 0.740
Correlaciones
calidad_code              1.000000
score_calidad             0.740022
eficiencia_km_l           0.526133
nivel_seguridad           0.497154
year                      0.495617
potencia_motor_hp         0.294303
transmission              0.197374
selling_price             0.103824
fuel                      0.031103
seller_type               0.028950
km_driven                -0.024826
tipo_carroceria          -0.033271
owner                    -0.045147
combustible_estimado_l   -0.107143
dtype: float64


##### Lo que esto nos dice
- La correlacion entre "score_calidad" y "calidad_auto" no es 1.0, lo que quiere decir que "calidad_auto" no depende al 100% de la variable "score_calidad".
- La variable "score_calidad" va a ser considerada.

#### Extraer marca del nombre
- Usar el nombre y cortarlo de una manera en la que podamos extrar la marca del auto para poder usarlo como variable y ser considerada.

In [245]:
# Función para extraer la marcaa
def extract_brand(df, carname_col='name'):
    """
    Extrae la marca ('brand') desde la columna carname_col en el DataFrame df.
    La marca es la primera palabra del nombre del auto, en minúsculas.

    Devuelve el mismo DataFrame con una columna nueva: 'brand'.
    """
    df['brand'] = df[carname_col].str.split(' ').str[0].str.lower()
    return df

# Aplicar la función
df = extract_brand(df)

# Mostrar las primeras filas con la nueva columna
print(df[['name', 'brand']].head())

# Ver las marcas únicas y su conteo
print("\nCantidad de marcas únicas:", df['brand'].nunique())
print(df['brand'].value_counts().head(29))

                       name    brand
0             Maruti 800 AC   maruti
1  Maruti Wagon R LXI Minor   maruti
2      Hyundai Verna 1.6 SX  hyundai
3    Datsun RediGO T Option   datsun
4     Honda Amaze VX i-DTEC    honda

Cantidad de marcas únicas: 29
brand
maruti           2413
hyundai          1724
mahindra          907
tata              897
honda             675
ford              669
toyota            546
chevrolet         422
volkswagen        326
renault           300
skoda             201
audi              164
nissan            164
fiat              140
mercedes-benz     109
bmw                93
datsun             93
land               25
jaguar             22
mitsubishi         20
volvo              18
ambassador         14
jeep               13
opelcorsa          10
kia                 9
mg                  8
daewoo              6
force               6
isuzu               6
Name: count, dtype: int64


#### Convertir columnas de enteros en categorias
- Columnas como "fuel", "seller_type", "transmission", "owner", "tipo_carroceria" tienen asignado un valor entero pero en realidad representan categorias sin orden.
- Convertir estas variables a Pandas' category dtype para mas claridad y eficiencia en la manipulacion de estas variables, tambien es mas seguro hacerlo de esta forma, asi evitamos que estas columnas se interpreten como numeric imputers o scalers.

In [246]:
cat_int_cols = [
    "fuel",
    "seller_type",
    "transmission",
    "owner",
    "tipo_carroceria",
    "brand"             # categoria extraida del nombre
]

for col in cat_int_cols:
    df[col] = df[col].astype("category")

#### Droppear columnas innecesarias
- Calcular correlaciones entre las columnas para identificar cual tiene un gran efecto en "calidad_auto"
- Definir X, y para entrenamiento

In [247]:
# Al final se droppea el nombre (ya habiendo extraido la brand)
df = df.drop(columns=["name"])

# Tirar la columna helper que definimos previemente
df = df.drop(columns=["calidad_code"])

print("\nColumnas que permanecen en el DataFrame:")
print(df.dtypes)  # Checar las columnas que permanecen

# Definir X & y para el training
features_to_drop = ["calidad_auto", "score_calidad", "combustible_estimado_l", "km_driven", "owner", "seller_type", "fuel" ]
# features_to_drop = ["calidad_auto", "score_calidad", "combustible_estimado_l", "km_driven", "owner", "seller_type", "fuel", "selling_price", "transmission"]
X = df.drop(columns=features_to_drop)
y = df["calidad_auto"]    # Estableciendo el target

# Inspeccionar las variables que permanecen
# print("Columnas a considerar:\n", X.columns.tolist())
print("\nFeature dtypes (para X):")
print(X.dtypes)


Columnas que permanecen en el DataFrame:
year                         int64
selling_price                int64
km_driven                    int64
fuel                      category
combustible_estimado_l     float64
seller_type               category
transmission              category
owner                     category
tipo_carroceria           category
potencia_motor_hp            int64
nivel_seguridad            float64
calidad_auto                object
score_calidad              float64
eficiencia_km_l            float64
brand                     category
dtype: object

Feature dtypes (para X):
year                    int64
selling_price           int64
transmission         category
tipo_carroceria      category
potencia_motor_hp       int64
nivel_seguridad       float64
eficiencia_km_l       float64
brand                category
dtype: object


### DummyClassifier & Baseline
- El primer paso es establecer una baseline y asegurarse de que la informacion esta limpia
- El objetivo es confirmar ques este DummyClassifier de como resultado una precision de 83.99%
- Se usara la estrategia de "most_frequent", simplemente el valor que mas se repite sera elegido para la prediccion.
- Este modelo representa el piso (baseline) de qualquier modelo futuro.
- El split para entrenamiento es un 80/20 asegurandose de que cada clase aparece con las misma proporciones.



In [248]:
# Establecer split (Train/Test)
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,    # 80/20 split
    stratify=y,       # preserva los ratios 84/10.5/5.5 para train y test
    random_state=42
)

# Test
# print("Train class proportions:\n", y_train.value_counts(normalize=True))
# print("\nTest class proportions:\n", y_test.value_counts(normalize=True))

# Iniciar DummyClassifier
dummy = DummyClassifier(
    strategy="most_frequent", # Siempre predice la clase mas frecuente ("Media")
    random_state=42
)

dummy.fit(X_train, y_train)     # "Entrena" basado en la clase mas frecuente

# Obtener predicciones
y_pred = dummy.predict(X_test)  # Aplica la regla a cada fila de X_test

# Evaluar accuracy
accuracy = accuracy_score(y_test, y_pred)   # Fraccion de predicciones correctas
print(f"DummyClassifier Accuracy: {accuracy}")

# Visualizar resulatdos de predicciones
print("Classification report:\n")
print(classification_report(
    y_test,
    y_pred,
    digits=4,         # Numero de decimales
    zero_division=0   # Ignorar zero-division errors (debido a strings)
))

# Generar confusion matrix
cm = confusion_matrix(y_test, y_pred, labels=["Media","Alta","Baja"])
print("Confusion Matrix:\n", cm)

DummyClassifier Accuracy: 0.84
Classification report:

              precision    recall  f1-score   support

        Alta     0.0000    0.0000    0.0000       210
        Baja     0.0000    0.0000    0.0000       110
       Media     0.8400    1.0000    0.9130      1680

    accuracy                         0.8400      2000
   macro avg     0.2800    0.3333    0.3043      2000
weighted avg     0.7056    0.8400    0.7670      2000

Confusion Matrix:
 [[1680    0    0]
 [ 210    0    0]
 [ 110    0    0]]


#### Lo que esto nos dice
- Tenemos un baseline de ~84% accuracy, este es el objetivo a vencer en los proximos modelos.

### Data preprocessing
- Preparar el data set para ser procesado
- Dividir entre variables numericas y categoricas
- Implementar un pipeline para cada tipo de variable
- Usar ColumnTransformer para aplicar los pipelines al DataFrame

#### Identificar columnas en base al dtype

In [249]:
# Datos numericos
num_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
print("Numeric features:", num_cols)

# Datos categoricos
cat_cols = X.select_dtypes(include=["category"]).columns.tolist()
print("\nCategorical features:", cat_cols)

Numeric features: ['year', 'selling_price', 'potencia_motor_hp', 'nivel_seguridad', 'eficiencia_km_l']

Categorical features: ['transmission', 'tipo_carroceria', 'brand']


#### Construir el pipeline numerico

In [250]:
num_pipe = Pipeline([
    # Definiendo tecnicas de defensive programming
    ("imputer", SimpleImputer(strategy="median")), # Imputacion de media
    ("scaler", StandardScaler())                   # Standard scaling
])

- El objetivo de la funcion `SimpleImputer()` es remplazar valores faltantes por el valor definido por su argumento (e.g. median).
- El objetivo de la funcion `StandardScaler()` es que despues de una imputacion, el valor numerico es escalado para tener una media = 0 y standard deviation = 1. De esta forma se evitan posibles errores generados por tener diferentes escalas de numeros en las diferentes columnas.

#### Construir el pipeline categorico

In [251]:
cat_pipe = Pipeline([
    # Definiendo tecnicas de defensive programming
    ("imputer", SimpleImputer(strategy="most_frequent")),  # Imputacion de moda
    ("encoder", OneHotEncoder(handle_unknown="ignore"))    # One-hot encoding
])

- `SimpleImputer()` remplaza cualquier valor faltante por el valor mas frecuente (la moda). Esto preserva la distribucion del data set.
- El objetivo del OneHotEncoder es convertir cada columna categorica en las columnas binarias necesarias para representar cada valor categorico unico.
  - `handle_unkwnown="ignore"` es util para que en caso de que aparezca una nueva categoria "vacia", que esta sea representada por una columna entera de puros ceros.

#### Implementacion de ColumnTransformer para aplicar los pipelines

In [252]:
preprocessor = ColumnTransformer(
    transformers=[
        # Aplicar num_pipe a las columnas numericas
        ("nums", num_pipe, num_cols),
        # Aplicar cat_pipe a columnas categoricas
        ("cats", cat_pipe, cat_cols)
    ],
    remainder="drop" # Tirar cualquier columna no listada
)

- `transformers` es un array de tuples `(name, pipeline, columns)` que ira siendo populado mientras `.fit()` corre
  1. `("nums", num_pipe, num_cols)` toma todas las columnas en `num_cols` y aplica el pipeline `num_pipe` y entraga como output el array numerico transformado.
  2. `("cats", cat_pipe, cat_cols)` toma todas las columnas en `cat_cols` y aplica el pipeline `cat_pipe` y entraga como output el array categorico transformado.

#### Inspeccionar el output transformado

In [253]:
# Testear el preprocessor
preprocessor.fit(X_train)

# Ver la forma del array final
X_train_transformed = preprocessor.transform(X_train)
print("Transformed shape:", X_train_transformed.shape)
#print(len(num_cols))

# Imprimir las columnas finales
feature_names = preprocessor.get_feature_names_out()
print("All output features:\n", feature_names)

Transformed shape: (8000, 41)
All output features:
 ['nums__year' 'nums__selling_price' 'nums__potencia_motor_hp'
 'nums__nivel_seguridad' 'nums__eficiencia_km_l' 'cats__transmission_0'
 'cats__transmission_1' 'cats__tipo_carroceria_1'
 'cats__tipo_carroceria_2' 'cats__tipo_carroceria_3'
 'cats__tipo_carroceria_4' 'cats__tipo_carroceria_5'
 'cats__brand_ambassador' 'cats__brand_audi' 'cats__brand_bmw'
 'cats__brand_chevrolet' 'cats__brand_daewoo' 'cats__brand_datsun'
 'cats__brand_fiat' 'cats__brand_force' 'cats__brand_ford'
 'cats__brand_honda' 'cats__brand_hyundai' 'cats__brand_isuzu'
 'cats__brand_jaguar' 'cats__brand_jeep' 'cats__brand_kia'
 'cats__brand_land' 'cats__brand_mahindra' 'cats__brand_maruti'
 'cats__brand_mercedes-benz' 'cats__brand_mg' 'cats__brand_mitsubishi'
 'cats__brand_nissan' 'cats__brand_opelcorsa' 'cats__brand_renault'
 'cats__brand_skoda' 'cats__brand_tata' 'cats__brand_toyota'
 'cats__brand_volkswagen' 'cats__brand_volvo']


##### Lo que esto nos dice
- En total hay 28 columnas ya con los pipelines aplicados (8 numericas y 20 categoricas) - temporal (falta agregar "brand" y talvez "submodel")
- Los prefijos `nums_` y `cats_` nos dicen de cual pipeline proviene cada columna
- Estos nombres nos serviran despues para implementacion de funciones o debugging

### Logistic Regression
- La regresión logística es una técnica de análisis de datos que utiliza las matemáticas para encontrar las relaciones entre dos factores de datos. Luego, utiliza esta relación para predecir el valor de uno de esos factores basándose en el otro. Normalmente, la predicción tiene un número finito de resultados, como un sí o un no.

In [254]:
# Crear un objecto de LogisticRegression
logreg = LogisticRegression(
    multi_class="multinomial",   # Clasificacion multiclase
    solver="saga",               # Algoritmo de optimizacion (soporta L1/L2)
    class_weight="balanced",     # Ajuste de pesos para clases desbalanceadas
    max_iter=1000,               # Numero maximo de iteraciones
    random_state=42
)

# Construir un unico pipeline
lr_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", logreg)
])

# Usar 5-fold cross-validation para evaluar el modelo de regresion logistica
cv_macro_f1 = cross_val_score(
    lr_pipeline,    # Pipeline completeo
    X,              # Todas las features
    y,              # Todas las labels
    cv=5,           # 5 folds
    scoring="f1_macro", # macro-averaged F1
    n_jobs=-1       # Usar todos los nucleos del CPU
)

# Imprimir los fold scores y la media
print("Logistic Regression 5-fold CV macro-F1 scores:", cv_macro_f1.round(4))
print("Mean macro-F1:", cv_macro_f1.mean().round(4))

Logistic Regression 5-fold CV macro-F1 scores: [0.9295 0.911  0.9005 0.8691 0.8722]
Mean macro-F1: 0.8964


##### Entendiendo la media Macro-F1
- La evaluacion multiclase Macro-F1 toma el promedio no ponderado de los tres puntajes F1 por clase ("Alta", "Media", "Baja"). Este proceso toma cada clase de forma equivalente aunque la distribucion sea diferente (10.5%, 84%, 5.5%). Usando macro-F1 nos aseguramos de que el modelo no esta eligiendo "Media" en todas las predicciones y realmente esta prediciendo entre las tres clases.
- El metodo de evaluacion 5-fold cross-validation es una manera de estimar que tan bien el pipeline (preprocessor + modelo) se generaliza a datasets no vistos
  - Se divide el dataset completo entre 5 subsets (folds), en este caso cada fold tiene aproximadamente 2000 carros.
  - Al final obtenemos 5 scores, una por cada subset, esto en toeria refleja que tan bien se comportaria el pipeline si fuera un "nuevo" dataset. El promedio de esas 5 scores es lo que llamamos mean macro-F1.

##### Lo que los resulatados del Macro-F1 nos dicen
- Los 5 scores estan arribe de 0.90, lo que quiere decir que el pipeline esta consistentemente obteniendo un score alto en "nueva" informacion.
- La media macro-F1 de 0.9274 es el mejor resumen numérico del "rendimiento esperado" si se entrenó con el 80 % del conjunto de datos y se probó con el 20 % restante. Esto sugiere que, en promedio, el modelo clasifica correctamente casi todos los ejemplos de "Media" y también funciona bien con "Alta" y "Baja".

#### Logistic Regression Training & Testing


In [255]:
# Test 1
# Establecer split (Train/Test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,    # 80/20 split
    stratify=y,       # preserva los ratios 84/10.5/5.5 para train y test
    random_state=42
)

# Asegurarse de las columnas presentes en X_train
print("Columns in X_train:", X_train.columns.tolist())

# Fit X_train, y_train (80%) y evaluar en X_test, y_test (20%)
lr_pipeline.fit(X_train, y_train)
y_test_pred = lr_pipeline.predict(X_test)

# Evaluar el test
test_acc = accuracy_score(y_test, y_test_pred)
print(f"Logistic Regression Test Accuracy: {test_acc:.4f}")
print("\nLogistic Regression Test Classification Report:\n")
print(classification_report(
    y_test,
    y_test_pred,
    digits=4,         # Numero de decimales
    zero_division=0   # Ignorar zero-division errors (debido a strings)
))

# Generar confusion matrix
cm = confusion_matrix(y_test, y_test_pred, labels=["Media","Alta","Baja"])
print("Confusion Matrix:\n", cm)

Columns in X_train: ['year', 'selling_price', 'transmission', 'tipo_carroceria', 'potencia_motor_hp', 'nivel_seguridad', 'eficiencia_km_l', 'brand']
Logistic Regression Test Accuracy: 0.9530

Logistic Regression Test Classification Report:

              precision    recall  f1-score   support

        Alta     0.8434    1.0000    0.9150       210
        Baja     0.6667    1.0000    0.8000       110
       Media     1.0000    0.9440    0.9712      1680

    accuracy                         0.9530      2000
   macro avg     0.8367    0.9813    0.8954      2000
weighted avg     0.9652    0.9530    0.9559      2000

Confusion Matrix:
 [[1586   39   55]
 [   0  210    0]
 [   0    0  110]]


In [256]:
# Test 2
# 3.3.1 New random split
X2_train, X2_test, y2_train, y2_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=99
)

# 3.3.2 Re‐fit your original pipeline (with score_calidad)
lr_pipeline.fit(X2_train, y2_train)
y2_pred = lr_pipeline.predict(X2_test)

acc2 = accuracy_score(y2_test, y2_pred)
print(f"New split (seed=99) Test Accuracy: {acc2:.4f}")
print("\nNew split Classification Report:\n")
print(classification_report(y2_test, y2_pred, digits=4, zero_division=0))

# Generar confusion matrix
cm = confusion_matrix(y2_test, y2_pred, labels=["Media","Alta","Baja"])
print("Confusion Matrix:\n", cm)

New split (seed=99) Test Accuracy: 0.9580

New split Classification Report:

              precision    recall  f1-score   support

        Alta     0.8434    1.0000    0.9150       210
        Baja     0.7097    1.0000    0.8302       110
       Media     1.0000    0.9500    0.9744      1680

    accuracy                         0.9580      2000
   macro avg     0.8510    0.9833    0.9065      2000
weighted avg     0.9676    0.9580    0.9602      2000

Confusion Matrix:
 [[1596   39   45]
 [   0  210    0]
 [   0    0  110]]


#### Testing Logistic Regression

In [257]:
# Input entry test
lr_pipeline.fit(X_train, y_train)

X.columns.tolist()

new_car_data = {
    "year": [2018],                   # int
    "selling_price": [350000],        # int or float
    "km_driven": [25000],             # int
    "fuel": [1],                      # must match the same category codes you used
    "combustible_estimado_l": [1800.0],  # float
    "seller_type": [0],               # category code
    "transmission": [1],              # category code
    "owner": [0],                     # category code
    "tipo_carroceria": [2],           # category code
    "potencia_motor_hp": [120],       # int
    "nivel_seguridad": [4.5],         # float
    "score_calidad": [5.5],           # float
    "eficiencia_km_l": [18.0],         # float
    "brand": ["Toyota"]             # string; will be cast to category below
}

new_car_df = pd.DataFrame(new_car_data)

cat_cols = ["fuel", "seller_type", "transmission", "owner", "tipo_carroceria", "brand"]
for col in cat_cols:
    new_car_df[col] = new_car_df[col].astype("category")

# new_car_df["brand"] = new_car_df["brand"].astype("category")

print(new_car_df.dtypes)

predicted_label = lr_pipeline.predict(new_car_df)
print("\nPredicted calidad_auto:", predicted_label[0])

predicted_proba = lr_pipeline.predict_proba(new_car_df)
print("Predicted probabilities [Alta, Media, Baja]:", predicted_proba[0])

year                         int64
selling_price                int64
km_driven                    int64
fuel                      category
combustible_estimado_l     float64
seller_type               category
transmission              category
owner                     category
tipo_carroceria           category
potencia_motor_hp            int64
nivel_seguridad            float64
score_calidad              float64
eficiencia_km_l            float64
brand                     category
dtype: object

Predicted calidad_auto: Alta
Predicted probabilities [Alta, Media, Baja]: [9.99999991e-01 1.38171057e-25 9.08181966e-09]


### Random Forest
- Random Forest es un algoritmo de aprendizaje supervisado en machine learning que utiliza múltiples árboles de decisión para clasificar o predecir datos. Es un método de conjunto (ensemble method) que mejora la precisión de las predicciones combinando los resultados de varios modelos débiles (árboles de decisión).

In [258]:
# Establecer split (Train/Test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,    # 80/20 split
    stratify=y,       # preserva los ratios 84/10.5/5.5 para train y test
    random_state=42
)

# Construir e inicializar un RandomForestClassifer
rf_clf = RandomForestClassifier(
    n_estimators=300,     # Numero de arboles en el bosque
    max_depth=None,       # Profundidad maxima de los arboles
    class_weight="balanced",  # Ajuste de pesos para clases desbalanceadas
    min_samples_split=2,  # Minimo numero de muestras requeridas para dividir
    random_state=42,      # Seed
    n_jobs=-1             # Usar todos los nucleos del CPU
)

# Crear un Pipeline unico que aplica el preprocessor previamente definido
rf_pipeline = Pipeline([
    # Usando el preprocessor que definimos previamente (ColumnTransformer)
    ("preprocessor", preprocessor),
    ("classifier", rf_clf)
])

- `n_estimators=300` construye 300 arbholes de decision
- `class_weight="balanced"` garantiza que las clases minoritarias tengan mayor ponderacion durante las divisiones
- `n_jobs=-1` paraleliza la construccion de arboles en todos los nucleos
- Reutilizamos el mismo preprocesador que gestiona la imputacion de la mediana, el escalado y el One-hot encoding. De esta forma, Random Forest ve la misma matriz numerica de 28 columnnas..

In [259]:
# Verificar las columnas consideradas
print("Features in X:", X.columns.tolist())

# if "score_calidad" in X.columns:
#     X = X.drop(columns=["score_calidad"])
# if "calidad_code" in X.columns:
#     X = X.drop(columns=["calidad_code"])

# Usar 5-fold cross-validation para evaluar el Random Forest
rf_cv_scores = cross_val_score(
    rf_pipeline,
    X,            # full feature set
    y,            # full labels
    cv=5,
    scoring="f1_macro",
    n_jobs=-1
)

# Imprimir scores del 5-fold test
print("Random Forest 5-fold CV macro-F1 scores:", rf_cv_scores.round(4))
print("Mean macro-F1 (RF):", rf_cv_scores.mean().round(4))

Features in X: ['year', 'selling_price', 'transmission', 'tipo_carroceria', 'potencia_motor_hp', 'nivel_seguridad', 'eficiencia_km_l', 'brand']
Random Forest 5-fold CV macro-F1 scores: [0.807  0.9036 0.9072 0.8987 0.8769]
Mean macro-F1 (RF): 0.8787


##### Lo que los resulatados del Macro-F1 nos dicen
- Los 5 scores son relativamente altos, lo que quiere decir que el pipeline esta consistentemente obteniendo un score alto en "nueva" informacion. Sinembargo, parece que el modelo es algo sensible dependeindo de cual 20% es escogido.
- Comparado a al modelo de Regresion Logistica, que tuvo un macro-F1 de 0.909 (ignorando la columna "score_calidad"), el modelo de Random Forest de hecho so comporta un poco peor en el 5-fold test. Esto puede significar que los limites lineales aprendidos por la regresion logistica eran mas precisos al momento de separar "Alta"/"Media"/"Baja" que el Random Forest.

#### Random Forest Training and Testing

In [260]:
# Fit Random Forest pipeline
rf_pipeline.fit(X_train, y_train)

# Predecir en el split de 20%
y_rf_pred = rf_pipeline.predict(X_test)

# Evaluar el modelo
rf_test_acc = accuracy_score(y_test, y_rf_pred)
print(f"Random Forest Test Accuracy: {rf_test_acc:.4f}")

# Classification report
print("\nRandom Forest Classification Report (Test Set):\n")
print(classification_report(
    y_test,
    y_rf_pred,
    digits=4,
    zero_division=0
))

# Confusion matrix
rf_cm = confusion_matrix(y_test, y_rf_pred, labels=["Media", "Alta", "Baja"])
print("\nRandom Forest Confusion Matrix (Test Set):\n", rf_cm)

Random Forest Test Accuracy: 0.9635

Random Forest Classification Report (Test Set):

              precision    recall  f1-score   support

        Alta     0.9620    0.8429    0.8985       210
        Baja     0.9639    0.7273    0.8290       110
       Media     0.9636    0.9940    0.9786      1680

    accuracy                         0.9635      2000
   macro avg     0.9632    0.8547    0.9020      2000
weighted avg     0.9635    0.9635    0.9620      2000


Random Forest Confusion Matrix (Test Set):
 [[1670    7    3]
 [  33  177    0]
 [  30    0   80]]


In [261]:
#Test 2
X2_train, X2_test, y2_train, y2_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=99
)
rf_clf2 = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    class_weight="balanced",
    random_state=42,
    n_jobs=-1
)
rf_pipeline2 = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", rf_clf2)
])
rf_pipeline2.fit(X2_train, y2_train)
y2_pred = rf_pipeline2.predict(X2_test)
acc2 = accuracy_score(y2_test, y2_pred)
print("New-split Test Accuracy:", acc2)
print(classification_report(y2_test, y2_pred, digits=4, zero_division=0))

# Confusion matrix
rf_cm = confusion_matrix(y2_test, y2_pred, labels=["Media", "Alta", "Baja"])
print("\nRandom Forest Confusion Matrix (Test Set):\n", rf_cm)

New-split Test Accuracy: 0.9565
              precision    recall  f1-score   support

        Alta     0.9503    0.8190    0.8798       210
        Baja     0.9375    0.6818    0.7895       110
       Media     0.9580    0.9917    0.9746      1680

    accuracy                         0.9565      2000
   macro avg     0.9486    0.8308    0.8813      2000
weighted avg     0.9561    0.9565    0.9544      2000


Random Forest Confusion Matrix (Test Set):
 [[1666    9    5]
 [  38  172    0]
 [  35    0   75]]


### Boosted Tree (LightGBM)
- LightGBM es un marco de trabajo de refuerzo de gradiente rápido, distribuido y de alto rendimiento que utiliza un algoritmo de aprendizaje basado en árboles. Está diseñado específicamente para grandes conjuntos de datos y datos de alta dimensión, y ofrece ventajas como mayor velocidad de entrenamiento, menor uso de memoria y mayor precisión en comparación con otros algoritmos de refuerzo. Los árboles reforzados, en general, son un método de aprendizaje conjunto que combina múltiples aprendices débiles (como árboles de decisión) para crear un modelo predictivo sólido.

In [262]:
# Definir un pipeline de LGBM
lgbm_clf = lgb.LGBMClassifier(
    objective="multiclass",
    class_weight="balanced",
    random_state=42,
    n_jobs=-1
)

lgbm_pipeline = Pipeline([
    # Usando el preprocessor que definimos previamente (ColumnTransformer)
    ("preprocessor", preprocessor),
    ("classifier", lgbm_clf)
])

# Usar 5-fold cross-validation para evaluar el LightGBM
lgbm_cv_scores = cross_val_score(
    lgbm_pipeline,
    X,
    y,
    cv=5,
    scoring="f1_macro",
    n_jobs=-1
)

# Imprimir scores del 5-fold test
print("LightGBM 5-fold CV macro-F1:", lgbm_cv_scores.round(4))
print("Mean macro-F1 (LGBM):", lgbm_cv_scores.mean().round(4))

LightGBM 5-fold CV macro-F1: [0.9414 0.941  0.9466 0.9301 0.9222]
Mean macro-F1 (LGBM): 0.9363


##### Lo que los resulatados del Macro-F1 nos dicen
- Los 5 scores son muy altos, lo que quiere decir que el pipeline esta consistentemente obteniendo un score alto en "nueva" informacion. El pipeline generaliza muy bien cada 20% del dataset
- Este modelo representa una mejora ante los dos modelos implementados previamente
  - Regresion Logistica (sin "score_calidad"): media CV macro-F1 ≈ 0.909
  - Random Forest (sin "score_calidad"): media CV macro-F1 ≈ 0.8613
- Hasta este punto el modelo de LightGBM tiene el mejor rendimiento categorizando el dataset y prediciendo "calidad_auto"

#### LightGBM testing

In [263]:
# Fit LGBM pipeline en el split de 80%
lgbm_pipeline.fit(X_train, y_train)

# Predecir en el split de 20%
y_lgbm_pred = lgbm_pipeline.predict(X_test)

# Evaluar el modelo
lgbm_test_acc = accuracy_score(y_test, y_lgbm_pred)
print(f"\nLightGBM Test Accuracy: {lgbm_test_acc:.4f}")

# Classification report
print("\nLightGBM Report (Test Set):\n")
print(classification_report(
    y_test,
    y_lgbm_pred,
    digits=4,
    zero_division=0
))

# Confusion matrix
lgbm_cm = confusion_matrix(y_test, y_lgbm_pred, labels=["Media", "Alta", "Baja"])
print("\nRandom Forest Confusion Matrix (Test Set):\n", lgbm_cm)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000430 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 872
[LightGBM] [Info] Number of data points in the train set: 8000, number of used features: 30
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612

LightGBM Test Accuracy: 0.9755

LightGBM Report (Test Set):

              precision    recall  f1-score   support

        Alta     0.9349    0.9571    0.9459       210
        Baja     0.8333    0.9545    0.8898       110
       Media     0.9916    0.9792    0.9853      1680

    accuracy                         0.9755      2000
   macro avg     0.9199    0.9636    0.9403      2000
weighted avg     0.9769    0.9755    0.9759      2000


Random Forest Confusion Matrix (Test Set):


### Boosted Tree (XGBoost)
- XGBoost es una potente biblioteca de aprendizaje automático de código abierto que utiliza el aumento de gradiente para crear modelos predictivos de alta precisión. Es conocida por su velocidad, eficiencia y capacidad para gestionar grandes conjuntos de datos. XGBoost es especialmente útil para tareas de clasificación y regresión, y se utiliza a menudo en competiciones de aprendizaje automático debido a su excelente rendimiento.

In [264]:
# Constuir un pipelien de XGBBoost
xgb_clf = XGBClassifier(
    objective="multi:softprob",   # objetivo multiclase
    num_class=3,                  # 3 clases: Alta, Media, Baja
    learning_rate=0.1,            # Tasa de aprendizaje
    n_estimators=200,             # Numero de arboles
    max_depth=6,                  # Profundidad de cada arbol
    subsample=0.8,                # Porcentaje de muestras
    colsample_bytree=0.8,         # Porcentaje de features
    scale_pos_weight=1,           # Balanceo de clases
    random_state=42,              # Seed
    n_jobs=-1                     # Usar todos los nucleos del CPU
)

xgb_pipeline = Pipeline([
    # Usando el preprocessor que definimos previamente (ColumnTransformer)
    ("preprocessor", preprocessor),
    ("classifier", xgb_clf)
])

# XGBoost requiere etiquetas en formato entero (no en string)
le = LabelEncoder()
y_encoded = le.fit_transform(y)
# Ahora y_encoded contiene {0, 1, 2} correspondiendo a ["Alta", "Baja", "Media"]

# Usar 5-fold cross-validation para evaluar XGBoost
xgb_cv_scores = cross_val_score(
    xgb_pipeline,
    X,
    y_encoded,
    cv=5,
    scoring="f1_macro",
    n_jobs=-1
)

# Imprimir scores del 5-fold test
print("XGBoost 5-fold CV macro-F1:", xgb_cv_scores.round(4))
print("Mean macro-F1 (XGBoost):", xgb_cv_scores.mean().round(4))

XGBoost 5-fold CV macro-F1: [0.9107 0.9561 0.9513 0.9329 0.932 ]
Mean macro-F1 (XGBoost): 0.9366


##### Explicación de los parametros
- `n_estimator=200` y `max_depth=6` son estandards de rango medio que funcionan con frecuencia.
- `subsample=0.8` y `colsample_bytree=0.8` agregan ensacado a nivel de fila y columna para reducir el sobreajuste.
- No pasamos directamente un `class_weight` a XGBClassifier, en su lugar, XGBoost usa `scale_pos_weight` solo para la clasficación binaria. Para multiclase lo dejamos en 1 y nos basamos en el muestreo balanceado mediante la division estratificada de la canalización.

##### Nota sobre la codificaión de etiquetas
- `LabelEncoder` toma las clases de tipo string `["Alta","Baja","Media"]` y las mapea a valores enteros `[0,1,2]` (por defecto en orden alfabetico: “Alta”→0, “Baja”→1, “Media”→2)
- `cross_val_score(..., y_encoded, ...)` satisface los requerimientos de XGBoost.

##### Lo que los resulatados del Macro-F1 nos dicen
- Al colocar aleatoriamente el 20 % de los 10,000 carros de cinco maneras diferentes, XGBoost promedia alrededor del 93,62 % de macro-F1, mejor que todos los modelos anteriores. Esto sugiere que XGBoost captura las sutiles interacciones no lineales entre las características con una eficacia ligeramente superior a la de LightGBM.
  - Logistic (sin `score_calidad`): ≈ 0.909
  - Random Forest (sin `score_calidad`): ≈ 0.8613
  - LightGBM (sin `score_calidad`): ≈ 0.9335
  - XGBoost (sin `score_calidad`): ≈ 0.9362

#### XGBoost testing

In [265]:
# Re-codificar los splits de entrenamiento
y_train_enc = le.transform(y_train)
y_test_enc  = le.transform(y_test)

# Fit XGBoost pipeline en el split de 80%
xgb_pipeline.fit(X_train, y_train_enc)

# Predecir en el split de 20%
y_xgb_pred_enc = xgb_pipeline.predict(X_test)
y_xgb_pred = le.inverse_transform(y_xgb_pred_enc) # Decodifica las predicciones

# Evaluar el modelo
xgb_test_acc = accuracy_score(y_test, y_xgb_pred)
print(f"XGBoost Test Accuracy: {xgb_test_acc:.4f}")

# Classification report
print("\nXGBoost Classification Report (Test Set):\n")
print(classification_report(
    y_test,
    y_xgb_pred,
    digits=4,
    zero_division=0
))

# Confusion matrix
xgb_cm = confusion_matrix(y_test, y_xgb_pred, labels=["Media", "Alta", "Baja"])
print("\nXGBoost Confusion Matrix (Test Set):\n", xgb_cm)

XGBoost Test Accuracy: 0.9800

XGBoost Classification Report (Test Set):

              precision    recall  f1-score   support

        Alta     0.9747    0.9190    0.9461       210
        Baja     0.9694    0.8636    0.9135       110
       Media     0.9812    0.9952    0.9882      1680

    accuracy                         0.9800      2000
   macro avg     0.9751    0.9260    0.9492      2000
weighted avg     0.9799    0.9800    0.9796      2000


XGBoost Confusion Matrix (Test Set):
 [[1672    5    3]
 [  17  193    0]
 [  15    0   95]]


#### Interpretando los resultados de XGBoost
Precisión general = 0,9770
- De los 2000 coches del split, XGBoost clasificó correctamente 1954 (≈ 97,7 %). Esto es superior al ≈ 97,55 % de LightGBM y al ≈ 96,15 % de Logistic Regression, asi como al Random Forest.

Precisión por clase / Recall / F1
1. “Alta” (210 ejemplos)
  - Precisión = 0,9742: De todos los carros que XGBoost predijo “Alta”, el 97,42 % eran realmente Alta (es decir, pocos falsos positivos de Alta).
  - Recall = 0,9000: El 90 % de los carros Alta reales se detectaron correctamente (es decir, el 10 % de los Alta se etiquetaron erróneamente como “Media”).
  - F1 = 0,9356: La media armónica que equilibra el recall de 0,90 y la precisión de 0,9742.

  - De la segunda fila de la matriz de confusión (`[ 21 189 0 ]`), XGBoost etiquetó correctamente 189 de los 210 carros Alta; 21 Alta se predijeron como Media (ninguno como Baja).

2. “Baja” (110 ejemplos)
  - Precisión = 0,9592: El 95,92 % de todo lo que se predijo como “Baja” en realidad era Baja.
  - Recall = 0,8545: Se detectó correctamente el 85,45 % de los 110 carros Baja reales; el 14,55 % restante (≈ 16 carros) se etiquetó erróneamente como Media.
  - F1 = 0,9038.
  - La tercera fila de la matriz de confusión (`[16 0 94]`): 94/110 Baja predijeron correctamente; 16 Baja se clasificaron erróneamente como Media.

3. “Media” (1680 ejemplos)
  - Precisión = 0,9783: De todos los vehículos etiquetados como “Media”, el 97,83 % lo fue realmente.
  - Recall = 0,9946: Se detectó correctamente el 99,46 % de los 1680 vehículos Media; solo ≈ 9 fueron mal etiquetados (5 como Alta, 4 como Baja).
  - F1 = 0,9864.
  En la primera fila de la matriz de confusión (`[1671 5 4]`): 1671/1680 Media predijeron correctamente, 5 se clasificaron erróneamente como Alta, 4 como Baja.

#### Hyperparameter Tunning for XGBoost

In [266]:
# Definir el espacio de busqueda de los hiperparametros
param_dist_xgb = {  # El estimador de XGBoost se llama "classifier"
    "classifier__learning_rate": [0.01, 0.05, 0.1],
    "classifier__n_estimators": [100, 200, 400],
    "classifier__max_depth": [4, 6, 8],
    "classifier__subsample": [0.6, 0.8, 1.0],
    "classifier__colsample_bytree": [0.6, 0.8, 1.0]
}

# Llevar a cabo una busqueda de hiperparametros randomizada en el diccionario
xgb_search = RandomizedSearchCV(
    xgb_pipeline,       # El estimador siendo tuneado
    param_dist_xgb,     # El diccionario que definimos
    n_iter=20,          # N diferentes combinaciones de parametros
    cv=3,               # Usar 3-fold cross validation (80/20/20)
    scoring="f1_macro", # Criteria a ser evaluada
    random_state=42,    # Seed
    n_jobs=-1           # Usar todos los nucleos del CPU
)

# Correr el randomized search (20 veces en total)
xgb_search.fit(X_train, y_train_enc)

# Imprimir los hiperparametros encontrados y los scores del CV
print("Best XGBoost params:", xgb_search.best_params_)
print("Best CV macro-F1 (XGB):", xgb_search.best_score_.round(4))

# Evaluar el modelo entero (preprocessor + XGBClassifier)
best_xgb = xgb_search.best_estimator_
y_tuned_pred_enc = best_xgb.predict(X_test)

# Decodificar resultados
y_best_xgb_str = le.inverse_transform(y_tuned_pred_enc)

# Imprimir macro-F1 score
test_macro_f1 = f1_score(y_test, y_best_xgb_str, average="macro")
print("Tuned XGBoost Test macro-F1:", test_macro_f1)

# Imprimir accuracy del modelo
tun_xgb_test_acc = accuracy_score(y_test, y_best_xgb_str)
print(f"XGBoost Test Accuracy: {tun_xgb_test_acc:.4f}")

# Classification report
print("\nTuned XGBoost Classification Report (Test Set):\n")
print(classification_report(
    y_test,
    y_best_xgb_str,
    digits=4,
    zero_division=0
))

# Confusion matrix
tun_xgb_cm = confusion_matrix(y_test, y_best_xgb_str, labels=["Media", "Alta", "Baja"])
print("\nTuned XGBoost Confusion Matrix (Test Set):\n", tun_xgb_cm)

Best XGBoost params: {'classifier__subsample': 0.8, 'classifier__n_estimators': 400, 'classifier__max_depth': 6, 'classifier__learning_rate': 0.1, 'classifier__colsample_bytree': 1.0}
Best CV macro-F1 (XGB): 0.9224
Tuned XGBoost Test macro-F1: 0.9575414311815337
XGBoost Test Accuracy: 0.9825

Tuned XGBoost Classification Report (Test Set):

              precision    recall  f1-score   support

        Alta     0.9797    0.9190    0.9484       210
        Baja     0.9615    0.9091    0.9346       110
       Media     0.9841    0.9952    0.9896      1680

    accuracy                         0.9825      2000
   macro avg     0.9751    0.9411    0.9575      2000
weighted avg     0.9824    0.9825    0.9823      2000


Tuned XGBoost Confusion Matrix (Test Set):
 [[1672    4    4]
 [  17  193    0]
 [  10    0  100]]


##### Entendiendo el hyperparameter tunning
- Los hiperparámetros son configuraciones externas que controlan cómo se entrena el modelo, pero no se aprenden de los datos.
  - Los ejemplos en XGBoost incluyen `learning_rate`, `n_estimators`, `max_depth`, `subsample` y `colsample_bytree`.
- Los hiperparámetros influyen considerablemente en el rendimiento de la generalización. Una mala elección (p. ej., `max_depth=1` o `learning_rate=1.0`) puede generar un underfit o un overfit.
- Los “mejores” valores de hiperparámetros dependen del conjunto de datos (tamaño, ruido, características de las funciones).
- `RandomizedSearchCV` muestrea aleatoriamente un número fijo (`n_iter`) de combinaciones de hiperparámetros de las distribuciones o listas especificadas. En la práctica, suele encontrar soluciones casi tan buenas como una búsqueda en cuadrícula completa, especialmente cuando solo se necesita una idea aproximada de la región óptima.

Significado de cada parametro definido en el espacio
- `"classifier__learning_rate": [0.01, 0.05, 0.1]`
  - Una tasa de aprendizaje más baja puede ralentizar el entrenamiento, pero a menudo produce una mejor generalización; una tasa más alta puede converger más rápido, pero conlleva el riesgo de overfit.
- `"classifier__n_estimators": [100, 200, 400]`
  - Número de rondas de refuerzo (árboles). Un mayor número de árboles puede capturar mayor complejidad, pero con un mayor coste computacional.
- `"classifier__max_depth": [4, 6, 8]`
  - Profundidad máxima de cada árbol. Los árboles más profundos pueden modelar interacciones más complejas, pero también pueden ocasionar overfit si son demasiado profundos.
- `"classifier__subsample": [0.6, 0.8, 1.0]`
  - Fracción de filas de entrenamiento utilizadas por cada árbol. Si la submuestra < 1,0, el algoritmo utiliza un 60 % u 80 % aleatorio de filas por árbol, lo que puede ayudar a reducir el overfit (como el bagging).
- `"classifier__colsample_bytree": [0.6, 0.8, 1.0]`
  - Fracción de características (columnas) consideradas por cada árbol. Valores más bajos obligan a cada árbol a ver solo un subconjunto de características, lo que puede mejorar la generalización de forma similar al bagging de características.


#### Retrain en el dataset completo

In [267]:
# Definir el pipeline final
final_xgb_pipeline = best_xgb

# Fit del pipeline en el dataset completo (10000 filas)
final_xgb_pipeline.fit(X, y_encoded)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('nums',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['year', 'selling_price',
                                                   'potencia_motor_hp',
                                                   'nivel_seguridad',
                                                   'eficiencia_km_l']),
                                                 ('cats',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(hand...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=6, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=400, n_jobs=-1, num_class=3,
                               num_parallel_tree=None, ...))])

- Este segmento se asegura de que el modelo de XGBoost vea cada carro del dataset completo.

### Deployment

#### Extraer la importancia de cada feature para visualización

In [268]:
# Extraer el XGBClassifier ya entrenado
xgb_model = final_xgb_pipeline.named_steps["classifier"]

# Extraer los nombres de las features
feature_names = final_xgb_pipeline.named_steps["preprocessor"].get_feature_names_out()

# Obtener las importancias de XGBoost
importances = xgb_model.feature_importances_

# Crear un DataFrame para visualizar las importancias
importance_df = pd.DataFrame({
    "feature": feature_names,
    "importance": importances
}).sort_values(by="importance", ascending=False)

# Mostrar las 10 features más importantes
print("Top 10 important features for XGBoost:\n")
print(importance_df.head(10))

Top 10 important features for XGBoost:

                    feature  importance
0                nums__year    0.158627
9   cats__tipo_carroceria_3    0.143178
11  cats__tipo_carroceria_5    0.073652
8   cats__tipo_carroceria_2    0.072318
4     nums__eficiencia_km_l    0.069292
3     nums__nivel_seguridad    0.066004
10  cats__tipo_carroceria_4    0.065356
2   nums__potencia_motor_hp    0.062546
7   cats__tipo_carroceria_1    0.046031
25         cats__brand_jeep    0.033672


#### Serializar el modelo y empezar despliegue

In [269]:
# Serializer el modelo para ser desplegado
joblib.dump(final_xgb_pipeline, "xgb_final_pipeline.pkl")

# Serializer el encoder para ser desplegado
joblib.dump(le, "label_encoder.pkl")

['label_encoder.pkl']

- Ahora cualquiera puede cargar el archivo "xgb_final_pipeline.pkl" y correr `.predict()` en data frames nuevos que tengan las mismas columnas y dtypes